In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import torch
import cv2
from itertools import groupby

In [ ]:
try:
    get_ipython().__class_._name__
    from tqdm.notebook import tqdm
except:
    from tqdm import tqdm

In [35]:
image_count = 612

In [36]:
def map_binary(mask):
    (threshold, black_and_white) = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    return black_and_white

In [37]:
def rle(binary_mask):
    rle = {'counts': [], 'size': list(binary_mask.shape)}
    counts = rle.get('counts')

    last_elem = 0
    running_length = 0

    for i, elem in enumerate(binary_mask.ravel(order='F')):
        if elem == last_elem:
            pass
        else:
            counts.append(running_length)
            running_length = 0
            last_elem = elem
        running_length += 1

    counts.append(running_length)

    return rle

In [43]:
data = {}
image_ids = []
encoded_pixels = []
for i in range(1, image_count): # TODO count over the actual images, rather than a range of numbers
    print(f'Encoding image {i} / {image_count}')
    img = cv2.imread(f'data/{i}.tif', 0)
    black_and_white = map_binary(img)
    rle_encoded = rle(black_and_white)
    image_ids.append(i)
    encoded_pixels.append(rle_encoded)

data['ImageId'] = image_ids
data['EncodedPixels'] = encoded_pixels
rle_df = pd.DataFrame(data, colums=['ImageId', 'EncodedPixels'])

Encoding image 1 / 612
Encoding image 2 / 612
Encoding image 3 / 612
Encoding image 4 / 612
Encoding image 5 / 612
Encoding image 6 / 612
Encoding image 7 / 612
Encoding image 8 / 612
Encoding image 9 / 612
Encoding image 10 / 612
Encoding image 11 / 612
Encoding image 12 / 612
Encoding image 13 / 612
Encoding image 14 / 612
Encoding image 15 / 612
Encoding image 16 / 612
Encoding image 17 / 612
Encoding image 18 / 612
Encoding image 19 / 612
Encoding image 20 / 612
Encoding image 21 / 612
Encoding image 22 / 612
Encoding image 23 / 612
Encoding image 24 / 612
Encoding image 25 / 612
Encoding image 26 / 612
Encoding image 27 / 612
Encoding image 28 / 612
Encoding image 29 / 612
Encoding image 30 / 612
Encoding image 31 / 612
Encoding image 32 / 612
Encoding image 33 / 612
Encoding image 34 / 612
Encoding image 35 / 612
Encoding image 36 / 612
Encoding image 37 / 612
Encoding image 38 / 612
Encoding image 39 / 612
Encoding image 40 / 612
Encoding image 41 / 612
Encoding image 42 / 612
E

Encoding image 337 / 612
Encoding image 338 / 612
Encoding image 339 / 612
Encoding image 340 / 612
Encoding image 341 / 612
Encoding image 342 / 612
Encoding image 343 / 612
Encoding image 344 / 612
Encoding image 345 / 612
Encoding image 346 / 612
Encoding image 347 / 612
Encoding image 348 / 612
Encoding image 349 / 612
Encoding image 350 / 612
Encoding image 351 / 612
Encoding image 352 / 612
Encoding image 353 / 612
Encoding image 354 / 612
Encoding image 355 / 612
Encoding image 356 / 612
Encoding image 357 / 612
Encoding image 358 / 612
Encoding image 359 / 612
Encoding image 360 / 612
Encoding image 361 / 612
Encoding image 362 / 612
Encoding image 363 / 612
Encoding image 364 / 612
Encoding image 365 / 612
Encoding image 366 / 612
Encoding image 367 / 612
Encoding image 368 / 612
Encoding image 369 / 612
Encoding image 370 / 612
Encoding image 371 / 612
Encoding image 372 / 612
Encoding image 373 / 612
Encoding image 374 / 612
Encoding image 375 / 612
Encoding image 376 / 612


NameError: name 'pd' is not defined

In [ ]:
class Dataset():
    def __init__(self, rle_df, image_base_dir, masks_base_dir, augmentation=None):
        self.df             = rle_df
        self.image_base_dir = image_base_dir
        self.masks_base_dir = masks_base_dir
        self.image_ids      = rle_df.ImageId.values
        self.augmentation   = augmentation
        
    def __getitem__(self, i):
        image_id  = self.image_ids[i]
        img_path  = os.path.join(self.image_base_dir, f'{image_id}.png')
        mask_path = os.path.join(self.masks_base_dir, f'{image_id}.png')
        image     = cv2.imread(img_path, 1)
        mask      = cv2.imread(mask_path, 0)
        
        if self.augmentation:
            sample = {'image': image, 'mask': mask}
            sample = self.augmentation(**sample)
            image, mask = sample['image'], sample['mask']
            
        return {
            'image': image,
            'mask': mask
        }
    
    def __len__(self):
        return len(self.image_ids)